In [ ]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time
import re

In [ ]:
def khan_crawling(start_date, end_date):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    start_date_ = "".join(start_date.split('.'))
    end_date_ = "".join(end_date.split('.'))
    eng_url_head = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q=kaist&sort=1&d1='
    kor_url_head = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q=카이스트&sort=1&d1='
    url_tail = str(start_date_)+'~'+str(end_date_)+'&pg='
    
    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 1
    while True:
        url_final = eng_url_head + url_tail + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.phArtc')) == 0:
                break
            elements = soup.select('.phArtc a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 1
    while True:
        url_final = kor_url_head + url_tail + str(page)
        try:  
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.phArtc')) == 0:
                break
            elements = soup.select('.phArtc a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
    url = list(set(url1).union(set(url2)))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link = getattr(row, 'url')
        response = requests.get(link, headers=headers)
        html = response.text
        soup = bs(html,'html.parser')
        try:
            title = soup.select('title')[0].get_text()
            content = soup.select('.art_body')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ")
            content = content.replace('\n', " ")
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            contents.append(content)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/"+str(start_date)+'-'+str(end_date)+"_kahn.xlsx",index = False)
    df.to_pickle("pickle files/"+str(start_date)+'-'+str(end_date)+"_kahn.pkl")

In [ ]:
khan_crawling('2015.01.01', '2020.10.23')

In [77]:
df = pd.read_pickle('pickle files/2015.01.01-2020.10.23_kahn.pkl')
df.head(10)

,urls,titles,contents
0,http://news.khan.co.kr/kh_news/khan_art_view.h...,‘새집증후군’ 유발 물질 잡는 정밀 센서 개발 - 경향신문,카이스트 김일두 교수팀 카이스트(KAIST) 연구진이 ‘새집증...
1,http://news.khan.co.kr/kh_news/khan_art_view.h...,카이스트에서 ‘인공지능 월드컵’ 연다,ㆍAI 경기 해설·기자 대회도 세계적으로 처음 시도되는 인공지능(AI) 월...
2,http://news.khan.co.kr/kh_news/khan_art_view.h...,KAIST '동전 절반 두께' 초박형 카메라 개발,한국과학기술원(KAIST) 연구진이 곤충 눈 구조를 본따 동전 두께 절반 정도의 ...
3,http://news.khan.co.kr/kh_news/khan_art_view.h...,[김상욱·유지원의 뉴턴의 아틀리에](23)자연과 생명의 완결된 복잡함을 미래의 조형...,복잡함 요리스 라르만 랩이 디자인한 대리석 소재의 의자(왼쪽)와 알...
4,http://news.khan.co.kr/kh_news/khan_art_view.h...,[송민령의 뇌과학이야기]우울에 빠진 뇌 - 경향신문,누구나 행복하기를 바란다. 까마득한 수천년 전부터 어떻게 살...
5,http://news.khan.co.kr/kh_news/khan_art_view.h...,카이스트 29일부터 ‘지능형 로봇워’ 본선대회 - 경향신문,카이스트(KAIST)는 오는 29일부터 다음달 1일까지 경기 고양시 일산 킨텍스 ...
6,http://news.khan.co.kr/kh_news/khan_art_view.h...,[기고]인공지능산업을 키우려면 - 경향신문,최근에 영국 공영방송 BBC는 한국이 인공지능산업에 가장 최적의 문화...
7,http://news.khan.co.kr/kh_news/khan_art_view.h...,오늘의 부고 - 2017년 9월 12일 - 경향신문,"■김창식 전 포항MBC 대표이사 별세, 정욱 SBI코스머니 대표이사·승욱 MBC스..."
8,http://news.khan.co.kr/kh_news/khan_art_view.h...,‘가슴 X레이’만으로 코로나19 진단하는 기술 개발 - 경향신문,"카이스트 연구팀, AI 활용 전 세계 코로나19 확진자가 500만명을 넘은..."
9,http://news.khan.co.kr/kh_news/khan_art_view.h...,[전문가의 세계 - 김상욱의 물리공부](11) 서로 모순되는 것이 공존하는 세계…양...,대립과 조화 서로 대립되는 두 명제 중 하나가 참이면 다른 하나는 ...
